In this experiment, we are going to visualize the uncertainty of a deep neural network by
using dropout as a mean of Bayesian Approximation [[1]](https://arxiv.org/abs/1506.02142).

We are going to use the [monkey species dataset from Kaggle](https://www.kaggle.com/slothkong/10-monkey-species).
If you want to run this notebook, clone it in your Colab, get your Kaggle API key,
upload it on your Colab, download the Kaggle monkey dataset and start cracking.

# Download Dataset from Kaggle

In [ ]:
! pip install -q kaggle
from google.colab import files

This is where you upload your Kaggle API JSON file.
Steps to do that: Go to your Kaggle account and look for API token generation section.
Expire your previous tokens (if you any of them) and then, create a new token. You can
just create a new token without expiring your previous tokens if you wish too.
This will download a `Kaggle.json` file. Upload this file in the following step.

In [ ]:
# Run this cell and press Browse to upload Kaggle.json file
files.upload()

### Download local copy of monkey dataset

In [ ]:
! mkdir ~/.kaggle
! mv kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d slothkong/10-monkey-species
! mkdir monkey_dataset
! unzip 10-monkey-species.zip -d monkey_dataset

In [ ]:
# Disable eager execution for K.function() with learning_phase to work properly
tf.compat.v1.disable_eager_execution()

# Preparation

In [1]:
from network import Network
from plotting import plot_grid, visualize_probdist
from dataset import Dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
IMAGE_SHAPE = (224, 224, 3)
BATCH_SIZE = 32

train_dir = 'monkey_dataset/training/training'
test_dir = 'monkey_dataset/validation/validation'

# The CSV file contains spaces for data fields and column names which we need to remove.
monkey_labels = pd.read_csv("monkey_dataset/monkey_labels.txt")
monkey_labels.columns = [c.strip() for c in monkey_labels.columns]
monkey_labels['Label'] = monkey_labels['Label'].str.strip()
monkey_labels['Common Name'] = monkey_labels['Common Name'].str.strip()

### Plot Training and Testing Data Count Distribution

In [ ]:
plt.figure(figsize=(10,4))

plt.bar(range(len(monkey_labels['Train Images'])), monkey_labels['Train Images'])
plt.bar(range(len(monkey_labels['Validation Images'])), monkey_labels['Validation Images'])
plt.xticks(range(len(monkey_labels['Common Name'])), monkey_labels['Common Name'], rotation=90)
plt.xlabel('Monkey Types')
plt.ylabel('Count')
plt.title('Training and Testing Data Distribution')
plt.show()

# Visualize Monkeys in the Dataset

In [ ]:
# [Class Folder Name]: [Common Name]

In [ ]:
plot_grid(rows=2, cols=5, figsize=(16,8),
          image_root_path=train_dir, labels=monkey_labels, data_shape=IMAGE_SHAPE[:2])

# Training

We want to train some classes of monkeys and see how our model performs for classes of monkeys which it was never trained before (testing out of distribution uncertainty). But this is not as extreme as testing out on completely different looking data samples (like objects, and other animals)

In [7]:
# Selecting classes of monkey to train
class_filter = ['n0', 'n1', 'n2', 'n3', 'n4']
label_mapping = monkey_labels['Common Name'][:5].to_dict() # Get class folder name -> common name mapping by taking the first five rows from dataframe

In [8]:
train_dataset = Dataset(path=train_dir, target_size=IMAGE_SHAPE[:2], dataset_type=Dataset.TRAIN,
                        batch_size=BATCH_SIZE,
                        class_filter=class_filter,
                        label_mapping=label_mapping)

val_dataset = Dataset(path=train_dir, target_size=IMAGE_SHAPE[:2], dataset_type=Dataset.VAL,
                        batch_size=BATCH_SIZE,
                        class_filter=class_filter,
                        label_mapping=label_mapping)

test_dataset = Dataset(path=test_dir, target_size=IMAGE_SHAPE[:2], dataset_type=Dataset.TEST,
                       batch_size=BATCH_SIZE,
                       class_filter=class_filter,
                       label_mapping=label_mapping)

Found 443 images belonging to 5 classes.
Found 0 images belonging to 5 classes.
Found 137 images belonging to 5 classes.


In [9]:
net = Network(input_shape=IMAGE_SHAPE, dropout_rate=0.2, num_classes=len(class_filter))

In [10]:
net.train_model(train_dataset, val_dataset, epochs=2)

Epoch 1/2
14/13 - 31s - loss: 1.7165 - accuracy: 0.2122
Epoch 2/2
14/13 - 29s - loss: 1.5480 - accuracy: 0.3341


In [23]:
y_prob_preds = net.model.predict(test_dataset.datagenerator)
y_preds = np.argmax(y_prob_preds, axis=1)

In [19]:
label_test = test_dataset.datagenerator.classes

In [21]:
from sklearn.metrics import classification_report
print(classification_report(label_test, y_preds))

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [22]:
y_preds

array([[0.39677206, 0.19933012, 0.14869651, 0.13312964, 0.1220717 ],
       [0.32058245, 0.20760886, 0.16841002, 0.17520942, 0.12818931],
       [0.39614293, 0.21668513, 0.1926044 , 0.09030802, 0.10425956],
       [0.40009305, 0.25548863, 0.20827076, 0.04939612, 0.08675144],
       [0.18949142, 0.18877243, 0.1554422 , 0.31760627, 0.14868772],
       [0.2047937 , 0.14222284, 0.1191541 , 0.4352787 , 0.09855054],
       [0.4470671 , 0.20086971, 0.1721299 , 0.06234588, 0.11758734],
       [0.3750311 , 0.2159591 , 0.2107952 , 0.0865964 , 0.11161818],
       [0.46867475, 0.19881521, 0.20819958, 0.02171258, 0.10259785],
       [0.41669363, 0.21390252, 0.19727474, 0.05752435, 0.11460468],
       [0.45374054, 0.23546465, 0.18817458, 0.03460193, 0.08801827],
       [0.21114376, 0.21474351, 0.17586592, 0.29236385, 0.10588299],
       [0.66144454, 0.11692709, 0.10132296, 0.06560969, 0.05469572],
       [0.4801554 , 0.18505664, 0.14675723, 0.09802983, 0.09000088],
       [0.13951156, 0.18626226, 0.